# Earthquake seismology with DAS

#### Todo List:
1. Reference earthquakes by Evid rather than dataframe index
2. Make a map of all of the detected events
3. Try some xcorr

In [1]:
import h5py
import matplotlib.pyplot as plt
from scipy.signal import butter, filtfilt
import numpy as np
import datetime
import pandas as pd
from dasquakes import sintela_to_datetime, open_sintela_file, local_earthquake_quicklook, data_wrangler, haversine
import obspy
from obspy import UTCDateTime
from obspy.clients.fdsn import Client
from libcomcat.search import search
from libcomcat.dataframes import get_summary_data_frame

ERROR 1: PROJ: proj_create_from_database: Open of /home/velgueta/.conda/envs/python-obspy/share/proj failed


### Get a list of events from USGS

In [2]:
events = search(starttime = datetime.datetime(2022, 3, 17, 0, 0), 
                endtime   = datetime.datetime.now(),
                minmagnitude=3.0,
                maxradiuskm=600, 
                latitude=47.60, 
                longitude=-122.33)

print("Returned %s events" % len(events))
 

Returned 30 events


### Choose an event to analyze

In [3]:
event_df = get_summary_data_frame(events)
event_df = event_df.sort_values(by=['time'],ascending=False)
event_df


#event_df.iloc[0::]

,id,time,location,latitude,longitude,depth,magnitude,alert,url,eventtype,significance
29,us6000jk08,2023-01-28 16:10:51.411,None,48.999000,-128.559200,10.000,4.00,None,https://earthquake.usgs.gov/earthquakes/eventp...,earthquake,247
28,us7000j1kj,2023-01-02 08:49:35.846,"228 km W of Bandon, Oregon",43.428900,-127.190900,10.000,3.90,None,https://earthquake.usgs.gov/earthquakes/eventp...,earthquake,234
27,us7000j3g5,2022-12-28 08:07:48.135,off the coast of Washington,47.841000,-128.912500,11.797,3.90,None,https://earthquake.usgs.gov/earthquakes/eventp...,earthquake,234
26,us7000j3f8,2022-12-28 08:06:54.980,"267 km SW of Tofino, Canada",47.804700,-128.908400,9.158,3.80,None,https://earthquake.usgs.gov/earthquakes/eventp...,earthquake,222
25,us7000ishe,2022-11-26 03:50:17.526,"Vancouver Island, Canada region",49.271400,-126.092300,33.380,4.90,None,https://earthquake.usgs.gov/earthquakes/eventp...,earthquake,527
24,us7000iqsk,2022-11-18 15:42:45.032,off the coast of Oregon,43.374100,-126.635600,10.000,4.40,None,https://earthquake.usgs.gov/earthquakes/eventp...,earthquake,298
23,us7000imha,2022-11-04 14:16:58.043,Off the coast of Oregon,43.401300,-127.099500,10.000,3.40,None,https://earthquake.usgs.gov/earthquakes/eventp...,earthquake,178
22,us7000im59,2022-11-03 09:57:33.841,Off the coast of Oregon,43.327300,-126.840700,10.000,3.50,None,https://earthquake.usgs.gov/earthquakes/eventp...,earthquake,189
21,us7000ikhh,2022-10-26 15:18:19.752,"169 km W of Langlois, Oregon",43.173500,-126.500500,10.000,3.20,None,https://earthquake.usgs.gov/earthquakes/eventp...,earthquake,158
20,us6000itsl,2022-10-14 21:23:02.037,"152 km WNW of Port Orford, Oregon",43.070100,-126.307200,10.000,3.30,None,https://earthquake.usgs.gov/earthquakes/eventp...,earthquake,168


In [ ]:
event_df.iterrows()

In [ ]:
#detected_quakes = ['uw61835766','uw61832097','uw61836427',
                 #  'uw61846212','uw61819867','uw61850651','uw61855012']
#detected_quakes_whidbey = ['nc73821036','uw61819867','uw61855012','us7000gxl1']

#this_event_index = 0
#this_id=detected_quakes[this_event_index]

#this_id = 'uw61832097'

#this_event = event_df[event_df.id==this_id]
#t0 = this_event['time'].iloc[0]

#this_event

In [ ]:
for index,this_event in event_df.iterrows():
    
    this_id = this_event['id']
    
    this_event_new = event_df[event_df.id==this_id]
    
    latitude_loc = this_event_new['latitude']
    longitude_loc = this_event_new['longitude']
    latitude_ref = 47.60
    longitude_ref = -122.33
    
    distance = haversine(longitude_loc,latitude_loc,longitude_ref,latitude_ref)
    
    if distance < 500:
        continue
    
    print(this_event['id'])
    
    stitle=f"M {this_event_new['magnitude'].to_numpy()[0]}, "\
           f"{this_event_new['location'].to_numpy()[0]}, "\
           f"{str(this_event_new['time'].to_numpy()[0])[0:-10]} UTC"
      

        
    cable = 'whidbey'
    record_length = 5 #minutes
    t0 = this_event_new['time'].iloc[0]
    
    prefix, network_name, datastore = data_wrangler(cable,record_length,t0)
        
    try:
        data,dates,attrs = open_sintela_file(prefix,
                                             t0,
                                             datastore,
                                             number_of_files=record_length,
                                             verbose=True)
        x_max=data.shape[1] * attrs['SpatialSamplingInterval']
        
        low_cut = 2
        hi_cut = 10

        b,a = butter(2,(low_cut,hi_cut),'bp',fs=attrs['MaximumFrequency']*2)
        data_filt = filtfilt(b,a,data,axis=0)
        
        filename = f"earthquakes_distances600km/"+this_id
        local_earthquake_quicklook(dates,20*data_filt,0,0,x_max,
                           stitle,filename=filename,das_vmax=0.02,
                           network_name=network_name,
                           skip_seismograms=True)
        timestamps = []
        for d in dates:
            if type(d) == datetime.datetime:
                timestamps.append(int((d.timestamp()-25200)*10**6))
            else:
                timestamps.append(d)
        g=h5py.File(filename+'.h5','w')
        g.create_dataset('datachannel',data=data_filt)
        g.create_dataset('time',data=np.array(timestamps))
        g.attrs.update(attrs)
        #g.create_dataset('attributes',data=str(attrs))
        g.close()
        
    except ValueError:
        continue
        # x_max=data.shape[1] * attrs['SpatialSamplingInterval']
        
   # low_cut = 2
    #hi_cut = 10

    #b,a = butter(2,(low_cut,hi_cut),'bp',fs=attrs['MaximumFrequency']*2)
    #data_filt = filtfilt(b,a,data,axis=0)
        
    #filename = f"earthquakes_figures"
    #local_earthquake_quicklook(dates,data_filt,x_max,
                      #     stitle,filename=filename,das_vmax=0.02,
                       #    network_name=network_name,
                        #   skip_seismograms=True)


us7000j1kj
Searching for files matching: /data/data6/whidbey/whidbey_2023-01-02_08-49*h5
Searching for files matching: /data/data6/whidbey/whidbey_2023-01-02_08-50*h5
Searching for files matching: /data/data6/whidbey/whidbey_2023-01-02_08-51*h5
Searching for files matching: /data/data6/whidbey/whidbey_2023-01-02_08-52*h5
Searching for files matching: /data/data6/whidbey/whidbey_2023-01-02_08-53*h5
us7000iqsk
Searching for files matching: /data/data6/whidbey/whidbey_2022-11-18_15-42*h5
Searching for files matching: /data/data6/whidbey/whidbey_2022-11-18_15-43*h5
Searching for files matching: /data/data6/whidbey/whidbey_2022-11-18_15-44*h5
Searching for files matching: /data/data6/whidbey/whidbey_2022-11-18_15-45*h5
Searching for files matching: /data/data6/whidbey/whidbey_2022-11-18_15-46*h5
us7000imha
Searching for files matching: /data/data6/whidbey/whidbey_2022-11-04_14-16*h5
us7000im59
Searching for files matching: /data/data6/whidbey/whidbey_2022-11-03_09-57*h5
Searching for files 

In [ ]:
attrs

In [ ]:
str(attrs)

In [ ]:
timestamps


In [ ]:
#calling 1 file saved to see how we can plot this.

#!ls earthquakes_figures
#file = 'earthquakes_figures/uw61832097.h5'
file = 'capstoneproject/uw61832097.h5'
data = h5py.File(file,'r')

In [ ]:
np.array(data['datachannel'])

len(np.array(data['datachannel']))


In [ ]:
len(np.array(data['time']))

In [ ]:
data.attrs.keys()

In [ ]:
data.attrs

In [ ]:
data.attrs['GaugeLength']

In [ ]:
data.attrs['SpatialSamplingInterval']

In [ ]:
data=np.array(data['datachannel'])


In [ ]:
data.shape

In [ ]:
data['datachannel']

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
#fig,ax=plt.subplots(figsize=(10,6))
#ax=plt.subplot(1,1,1)
date_format = mdates.DateFormatter('%H:%M:%S')
x_lims = mdates.date2num(np.array(data['time']))
x_max = np.array(data['datachannel']).shape[1] * data.attrs['SpatialSamplingInterval']
fig,ax=plt.subplots(figsize=(10,3))
plt.imshow(np.array(data['datachannel']).T,cmap='seismic',aspect='auto',vmin=-0.02,vmax=0.02,extent=[x_lims[0],x_lims[-1],x_max,0])
plt.title('Simple image of das')
ax.xaxis.set_major_formatter(date_format)
ax.xaxis_date()
plt.grid()
#plt.xlabel('Channel Number', fontsize=5)
#plt.ylabel('Time Step', fontsize=5)
#plt.colorbar()
plt.show()

In [ ]:
#Below are just some tests; nothing relevant to the code above

In [ ]:
x_lims

In [ ]:
x_lims[0]

In [ ]:
x_lims[-1]

In [ ]:
np.array(data['time'])

In [ ]:
np.array(data['time'])[-1]-np.array(data['time'])[0]

In [ ]:
for d in dates[5980:6001]:
    print((d.timestamp()-25200)*10**6)
for d in dates[6001:6020]:
    print(d)

In [ ]:
(dates[-18]-dates[0].timestamp()*10**6)/10**6

In [ ]:
dates[:]

In [ ]:
len(dates)

In [ ]:
100*60

In [ ]:
dates[5980:6020]

In [ ]:
dates[:10]

In [ ]:
dates[:20]

In [ ]:
for d in dates[11980:12020]:
    print(d)

In [ ]:
mdates.date2num(dates)[5980:6020]

In [ ]:
mdates.date2num(dates)[-1]-mdates.date2num(dates)[0]

In [ ]:
(np.array(data['time'])[-1]-np.array(data['time'])[0])

In [ ]:
np.round(599990004*mdates.date2num(datesaux)/(mdates.date2num(datesaux)[-1]-mdates.date2num(datesaux)[0])).astype(int)

In [ ]:
np.array(data['time'])

In [ ]:
this_id = 'us6000hn38'

this_event_new = event_df[event_df.id==this_id]
stitle=f"M {this_event_new['magnitude'].to_numpy()[0]}, "\
       f"{this_event_new['location'].to_numpy()[0]}, "\
       f"{str(this_event_new['time'].to_numpy()[0])[0:-10]} UTC"
      
cable = 'whidbey'
record_length = 10 #minutes
t0 = this_event_new['time'].iloc[0]
    
prefix, network_name, datastore = data_wrangler(cable,record_length,t0)
        
dataaux,datesaux,attrsaux = open_sintela_file(prefix,
                                             t0,
                                             datastore,
                                             number_of_files=record_length,
                                             verbose=True)

In [ ]:
for i in range(0,10,3):
    print(i)

In [ ]:
13/10

In [ ]:
event_df.keys()